In [1]:
# Redmine ticket 6870 - Siblings auto-complete, for all visits completed.
# Imports
import json
import os 
import pandas as pd
from django.apps import apps as django_apps
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
import pytz
from copy import copy
from django.db.models.signals import post_save
from django.dispatch import receiver
from edc_appointment.models import Appointment
from edc_appointment.constants import NEW_APPT, INCOMPLETE_APPT
from edc_metadata.constants import REQUIRED, KEYED
from django.db.models.deletion import ProtectedError
from flourish_caregiver.helper_classes.auto_complete_child_crfs import AutoCompleteChildCrfs
tz = pytz.timezone('Africa/Gaborone')

In [3]:
def create_new_visit_objs(visit_obj, completed_visit):
    helper_cls = AutoCompleteChildCrfs(visit_obj, completed_visit)
    helper_cls.pre_fill_crfs()

In [4]:
def disconnect_visit_post_save():
    post_save.disconnect(sender=MaternalVisit, dispatch_uid='maternal_visit_on_post_save')
disconnect_visit_post_save()

In [5]:
sidx = ['B142-040990713-5', 'B142-040990466-0', 'B142-040990612-9', 'B142-040990837-2', 'B142-040990857-0',
        'B142-040990022-1', 'B142-040990666-5', 'B142-040990562-6', 'B142-040990705-1', 'B142-040990570-9',
        'B142-040990804-2', 'B142-040990786-1', 'B142-040990558-4', 'B142-040990023-9', 'B142-040990355-5',
        'B142-040990669-9', 'B142-040990099-9', 'B142-040990528-7', 'B142-040990172-4', 'B142-040991009-7',
        'B142-040990044-5', 'B142-040990116-1', 'B142-040990639-2', 'B142-040990893-5']

In [6]:
for sid in sidx:
    completed_quart_appts = Appointment.objects.filter(
        subject_identifier=sid, schedule_name__icontains='quart').exclude(
        appt_status=NEW_APPT).values_list('visit_code', flat=True).distinct()
    completed_quart_appts = list(set(completed_quart_appts))
    for visit_code in completed_quart_appts:
        multi_visits = MaternalVisit.objects.filter(
            subject_identifier=sid, visit_code=visit_code, visit_code_sequence=0).order_by('created')
        if multi_visits.count() > 1:
            initial_visit = multi_visits[0]
            latest_visit = multi_visits[1]

            print('Participant ID', sid, 'Visit code', visit_code, 'Schedule name', latest_visit.schedule_name)
            crf_metadatas = CrfMetadata.objects.filter(
                subject_identifier=sid, visit_code=visit_code, visit_code_sequence=0,
                schedule_name=latest_visit.schedule_name, entry_status=KEYED)

            for crf_metadata in crf_metadatas:
                crf_cls = django_apps.get_model(crf_metadata.model)
                try:
                    crf_obj = crf_cls.objects.get(maternal_visit=latest_visit)
                except crf_cls.DoesNotExist:
                    print('Why do you have keyed metadata 🤔')
                    crf_metadata.entry_status = REQUIRED
                    crf_metadata.save()
                    continue
                else:
                    crf_obj.delete()

            appt = Appointment.objects.get(subject_identifier=sid, visit_code=visit_code,
                                           visit_code_sequence=0, schedule_name=latest_visit.schedule_name)

            try:
                latest_visit.delete()
            except ProtectedError:
                print('Do not do that 😒')
                raise
            else:
                appt.appt_datetime = appt.timepoint_datetime
                appt.appt_status = NEW_APPT
                appt.save()


In [7]:
for sid in sidx:
    completed_quart_appts = Appointment.objects.filter(
        subject_identifier=sid, schedule_name__icontains='quart').exclude(appt_status=NEW_APPT)
    for appt in completed_quart_appts:
        appt_dt = appt.appt_datetime
        try:
            completed_visit = MaternalVisit.objects.get(appointment=appt)
        except MaternalVisit.DoesNotExist:
            continue
        else:
            new_appt = None
            new_appts = Appointment.objects.filter(
                subject_identifier=sid,
                schedule_name__icontains='quart',
                visit_code_sequence=appt.visit_code_sequence,
                appt_status=NEW_APPT).exclude(schedule_name=appt.schedule_name, ).order_by('timepoint_datetime')
            
            for new_ap in new_appts:
                visit_definition = new_ap.visits.get(new_ap.visit_code)
                ideal_timepoint = new_ap.timepoint_datetime
                
                earliest_appt_dt = (ideal_timepoint - visit_definition.rlower).astimezone(tz)
                latest_appt_dt = (ideal_timepoint + visit_definition.rupper).astimezone(tz)
        
                if earliest_appt_dt <= appt_dt and latest_appt_dt >= appt_dt:
                    new_appt = new_ap
                    break

            if new_appt:
                try:
                    new_visit = MaternalVisit.objects.get(appointment=new_appt)
                except MaternalVisit.DoesNotExist:
                    new_visit = copy(completed_visit)
                    new_visit.id = None
                    new_visit.schedule_name = new_appt.schedule_name
                    new_visit.visit_schedule_name = new_appt.visit_schedule_name
                    new_visit.appointment = new_appt
                    new_visit.save()

                    crf_metadata = CrfMetadata.objects.filter(
                        subject_identifier=sid, visit_code=new_visit.visit_code,
                        visit_code_sequence=new_visit.visit_code_sequence,
                        schedule_name=new_visit.schedule_name, )

                    crf_metadata.delete()
                    metadata = new_visit.metadata_cls(visit=new_visit, update_keyed=False)
                    metadata.prepare()
    
                    create_new_visit_objs(new_visit, completed_visit)
                else:
                    crf_metadata = CrfMetadata.objects.filter(
                        subject_identifier=sid, visit_code=new_visit.visit_code,
                        visit_code_sequence=new_visit.visit_code_sequence,
                        schedule_name=new_visit.schedule_name, )

                    crf_metadata.delete()
                    metadata = new_visit.metadata_cls(visit=new_visit, update_keyed=False)
                    metadata.prepare()
    
                    create_new_visit_objs(new_visit, completed_visit)

                # change new appointment status to incomplete
                new_appt.appt_datetime = appt.appt_datetime
                new_appt.appt_status = INCOMPLETE_APPT
                new_appt.appt_reason = appt.appt_reason
                new_appt.save()
    